In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

### Load phrase base

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/monotone_base.pickle')

### Draw notations

In [ ]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [ ]:
abc_notes = BitNotes(note_count=26, 
                     notation_count=5, 
                     active_bits=8, 
                     bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

In [ ]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

### Learn model

In [3]:
def feed_phrase_batch(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200, random_bit_key=False)
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = bit_notes.phrase_chord_as_bits(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)

def feed_n_batches(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, n=10, batch_size=200):
    for i in range(n):
        feed_phrase_batch(cxnn, feeder, bit_notes, count=batch_size)
        print(f'batch {i+1}/{n}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def learn_cycle(cxnn: ContextNN, 
                feeder: PhraseFeeder, 
                bit_notes: BitNotes,
                batch_size=200,
                n_accumulate_batches=30,
                n_consolidate_batches=50,
                reduce_min_component=0.2,
                reduce_min_activations=100):
    print('\naccumulating clusters...')
    cxnn.state = const.STATE_ACCUMULATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_accumulate_batches, batch_size=batch_size)
    
    print('\nconsolidating clusters...')
    cxnn.state = const.STATE_CONSOLIDATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_consolidate_batches, batch_size=batch_size)
    
    print('\nreducing clusters...')
    cxnn.reduce_clusters(min_component=reduce_min_component, 
                         min_activations=reduce_min_activations,
                         trim=True,
                         remain_parts=3,
                         clear_stats=True,
                         consolidate=True,
                         amnesty=True)
    print(f'clusters: {cxnn.cluster_count()}')
    

In [7]:
notes = bitnotes.BitNotes(note_count=26, 
                          notation_count=5, 
                          active_bits=8, 
                          bit_count=255)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=255,
                            output_bit_count=len(marks.keys()),
                            watch_point_count=500,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4,
                            bit_notes=notes,
                            data_marks=marks)

In [8]:
%%time
for n in range(10):
    print(f'\nCycle {n+1}')
    learn_cycle(cxnn, 
                feeder, 
                notes,
                batch_size=200,
                n_accumulate_batches=30,
                n_consolidate_batches=30,
                reduce_min_component=0.2,
                reduce_min_activations=100)


Cycle 1

accumulating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batch 14/30
batch 15/30
batch 16/30
batch 17/30
batch 18/30
batch 19/30
batch 20/30
batch 21/30
batch 22/30
batch 23/30
batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 77564

consolidating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/30
batch 13/30
batch 14/30
batch 15/30
batch 16/30
batch 17/30
batch 18/30
batch 19/30
batch 20/30
batch 21/30
batch 22/30
batch 23/30
batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 77564

reducing clusters...
clusters: 4330

Cycle 2

accumulating clusters...
batch 1/30
batch 2/30
batch 3/30
batch 4/30
batch 5/30
batch 6/30
batch 7/30
batch 8/30
batch 9/30
batch 10/30
batch 11/30
batch 12/

batch 24/30
batch 25/30
batch 26/30
batch 27/30
batch 28/30
batch 29/30
batch 30/30
clusters: 105089

reducing clusters...
clusters: 29822
Wall time: 12min 54s


### Model stats

In [9]:
print(cxnn.cluster_count())
print(sorted(cxnn.point_stats(), key=lambda x: x[0]))
# print(cxnn.point_stats())
marks

29822
[(0, 59), (0, 4), (0, 73), (0, 0), (0, 16), (0, 98), (0, 180), (0, 304), (0, 209), (0, 756), (0, 1), (0, 9), (0, 464), (0, 2), (0, 42), (0, 331), (0, 0), (0, 2), (0, 39), (0, 0), (0, 4), (0, 494), (0, 5), (0, 134), (0, 1), (0, 0), (0, 465), (0, 105), (0, 0), (0, 26), (0, 0), (0, 0), (0, 39), (0, 57), (0, 443), (0, 0), (0, 9), (0, 164), (0, 171), (0, 0), (0, 0), (0, 7), (0, 4), (0, 20), (0, 474), (0, 22), (0, 469), (0, 0), (0, 43), (0, 3), (0, 63), (0, 6), (0, 14), (0, 0), (0, 23), (0, 4), (0, 132), (0, 0), (0, 425), (0, 213), (0, 9), (0, 23), (0, 3), (1, 155), (1, 52), (1, 3), (1, 85), (1, 54), (1, 2), (1, 8), (1, 2), (1, 25), (1, 0), (1, 220), (1, 1), (1, 199), (1, 23), (1, 25), (1, 1), (1, 10), (1, 15), (1, 220), (1, 6), (1, 75), (1, 33), (1, 30), (1, 9), (1, 373), (1, 0), (1, 155), (1, 51), (1, 0), (1, 272), (1, 6), (1, 0), (1, 15), (1, 14), (1, 283), (1, 180), (1, 174), (1, 37), (1, 67), (1, 52), (1, 8), (1, 7), (1, 11), (1, 68), (1, 10), (1, 5), (1, 35), (1, 1), (1, 8), (1, 

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [15]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

17496
{7: 272, 8: 6849, 9: 9992, 10: 378, 11: 5}
{0: 17496}


In [16]:
cxnn.cluster_consolidated_stats()

{0: 1021, 1: 4592, 2: 3397, 3: 3065, 4: 2683, 5: 2738}

In [ ]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = cxnn.point_objects[8]
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

print(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(np.where(cluster.bit_mask > 0)[0])
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

In [ ]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

if cluster_with_stats:
    print(cluster_with_stats.stats, cluster_with_stats.consolidated)
else:
    print('nothing to print')
    
    

### Save-load model

In [11]:
import pickle

def save_cxnn(cxnn: ContextNN, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(cxnn, f)

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def save_notes(notes: BitNotes, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(notes, f)

def load_notes(file_name: str) -> BitNotes:
    with open(file_name, 'rb') as f:
        notes = pickle.load(f)
        return notes



In [12]:
save_cxnn(cxnn, './data/cxnn_langs.pickle')

In [ ]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')

In [ ]:
save_cxnn(cxnn, './data/cxnn_monotone.pickle')